# Plot TF output

We'll just use the stat file to make pretty plots  of the output from the TF runs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('plots.mplstyle')

from glob import glob
from collections import OrderedDict

Import TerraFERMA and ThermoCodegen related objects

In [ ]:
from buckettools.statfile import parser

In [ ]:
import py_fo_sio2_poly_linear_rxns as pfosi
rxn = pfosi.fo_sio2_poly_linear_rxns()
rxn.report()

Extract Phase abbreviations and Liquid endmember names and set up indexing dict

In [ ]:
phases = rxn.phases()
abbrevs = [ phase.abbrev() for phase in phases]
ip = OrderedDict(zip(abbrevs,range(len(abbrevs))))
enames = [phases[ip['Lq']].endmember_name(i) for i in range(phases[ip['Lq']].endmember_number())]
print('Phase Abbreviations = {}'.format(abbrevs))
print('Liquid Endmembers =  {}'.format(enames))

In [ ]:
def plotTf(stat_file):
    stat = parser(stat_file)
    """ Function to extract data from stat files and create a pdf plot"""
    # Extract data
    P = stat['IsentropicDecompression']['Pressure']['max']
    T = stat['IsentropicDecompression']['Temperature']['max']
    M = stat['IsentropicDecompression']['Mass']['max']
    cLq = stat['IsentropicDecompression']['cLq']['max']
    Phi = stat['IsentropicDecompression']['Phi']['max']
    S0 = stat['IsentropicDecompression']['S0']['max'][0]
    WallTime = stat['ElapsedWallTime']['value'][-1]
    dt0 = stat['dt']['value'][0]

    # calculate total Entropy and relative error from S0
    S = np.zeros(len(T))
    C = rxn.zero_C()
    for n in range(1,len(C)):
        C[n] = [1.]

    for t in range(len(T)):
        C[0] = cLq[:,t]
        S[t] = np.array(rxn.s(T[t],P[t],C)).dot(M[:,t])
    
    # de-nan and calculate relative error in entrop
    err = np.abs(S-S0)/np.abs(S0)
    nans = np.isnan(err)
    err = err[ ~nans]

    
    # plot the bugger out with pressure in Kb
    P = P/1000.
    
    fig  = plt.figure(figsize=(20,8))
    ax = fig.add_subplot(1,4,1)

    for i in range(len(M)):
        ax.plot(M[i],P,label=abbrevs[i])
    ax.set_ylim(P[0],P[-1])
    ax.set_ylabel('P (kb)')
    ax.set_xlabel('Mass (g)')
    #ax.legend(loc=(.65,.76))
    ax.legend(loc='best')
    ax.grid()
    ax.set_title('$S_0$ ={:3.3f} J/K'.format(S0))


    ax = fig.add_subplot(1,4,2)
    for i in range(len(Phi)):
        ax.plot(Phi[i],P,label=abbrevs[i])
    ax.set_ylim(P[0],P[-1])
    ax.set_ylabel('P (kb)')
    ax.set_xlabel('Phase Fractions ($\Phi$)')
    #ax.legend(loc=(.65,.76))
    ax.legend(loc='best')
    ax.set_title('Solver = TF Wall={:3.3f} s, N={} steps'.format(WallTime,len(P)))
    ax.grid()


    ax = fig.add_subplot(1,4,3)
    for k in range(len(cLq)):
        ax.plot(cLq[k],P,label=enames[k])
    ax.set_ylim(P[0],P[-1])
    ax.set_ylabel('P (kb)')
    ax.set_xlabel('C (wt %)')
    ax.set_xlim(-0.01, 1.0)
    ax.legend(loc='lower center')
    ax.grid()


    #ax.set_title('Isentropic decompression S_0 ={:3.3f} J/K'.format(S0))
    ax = fig.add_subplot(1,4,4)
    ax.plot(T-273.15, P, 'b-')
    ax.set_xlabel('T (C)')
    ax.set_ylabel('P (kb)')
    ax.set_ylim(P[0],P[-1])
    ax.set_title('Temperature, $||(S-S_0)/S_0||_2$ = {:3.3g}'.format(np.linalg.norm(err)))
    ax.grid()

    # need a better title or save to each output directory...not sure yet
    fig.suptitle('Isentropic Decompression $S_0$ ={:3.3f} J/K'.format(S0))
    plt.savefig('Isentropic_TF_dt{}.pdf'.format(dt0))
    plt.show()

In [ ]:
stat_files = glob('*.run/**/*.stat',recursive=True)
for file in stat_files:
    plotTf(file)